# Combine flare lists from different instruments

The purpose of this script is to create and test functions that are instrument-specific and allow to import a dataframe containing flare lists, peak times, and flare locations. The code outside of these functions should run independently of the instrument used, so that it can be compared with STIX.

[Status 30-Jun-2023]: The functions for coordinate transformations may be a bit outdated, as I created them almost two years ago. However, they still work (I updated my library recently and they seem to still function properly). Therefore, for the first version, I suggest we use these functions. We can update them later if necessary.

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import spiceypy as spice

from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy import units as u
from sunpy.coordinates import Helioprojective
from sunpy.coordinates.frames import HeliocentricEarthEcliptic, HeliographicStonyhurst

# Do not show warnings
import warnings
warnings.filterwarnings('ignore')

### Functions for dealing with the SPICE kernel

In [2]:
def load_SPICE(path_kernel):
    """
    Load the SPICE kernel that will be used to get the
    coordinates of Solar Orbiter
    """

    #get cwd
    cwd=os.getcwd()

    # Check if path_kernel has folder format
    if path_kernel[-1] != '/':
        path_kernel = path_kernel+'/'

    # Change the CWD to the given path. Necessary to load correctly all kernels
    os.chdir(path_kernel)

    # Load one (or more) SPICE kernel into the program
    spice_kernel = 'solo_ANC_soc-flown-mk.tm'
    spice.furnsh(spice_kernel)

    print()
    print('SPICE kernels loaded correctly')
    print()

    #change back to original working directory
    os.chdir(cwd)

In [3]:
def so2hgs(coord, date, solo_hee):
    """
    Takes the coordinates of the region of interest (ROI) as
    seen from Solar Orbiter (HPC Solar Orbiter frame) and
    transform them to hg Stonyhurst coordinates
    """

    # SkyCoord of the ROI as seen from Solar Orbiter
    roi_solo_hpc = SkyCoord(coord[0]*u.arcsec, 
                            coord[1]*u.arcsec, 
                            #frame=map.coordinate_frame,
                            obstime=date,
                            observer=solo_hee.transform_to(HeliographicStonyhurst(obstime=date)),
                            frame='helioprojective')
    
    # Assume ROI to be on the surface of the Sun
    roi_inter = roi_solo_hpc.transform_to(HeliocentricEarthEcliptic)
    third_dim = 1*u.Rsun

    # ROI location in HEE
    roi_hee = SkyCoord(roi_inter.lon, roi_inter.lat, third_dim, 
                       frame=HeliocentricEarthEcliptic(obstime=date))

    # Since now we have the full 3D coordinate of the ROI position
    # given in HEE, we can now transform that coordinated as seen
    # from Solar Orbiter and give them in hg Stonyhurst coordinates
    roi_hgc_stony = roi_hee.transform_to(HeliographicStonyhurst(obstime=date))
    
    return roi_hgc_stony

In [4]:
def get_earth_hee(date_earth):
    """
    Get the coordinates of the Earth and then return them in
    Heliocentric Earth Ecliptic (HEE) coordinates.
    """

    # Observing time (to get the Earth coordinates)
    et_stix = spice.datetime2et(date_earth)

    # Obtain the coordinates of Solar Orbiter
    earth_hee_spice = spice.spkpos('EARTH', et_stix,
                                     'SOLO_HEE_NASA', #  Reference frame of the output position vector of the object 
                                     'NONE', 'SUN')[0]
    earth_hee_spice = earth_hee_spice * u.km

    # Convert the coordinates to HEE
    earth_hee = HeliocentricEarthEcliptic(earth_hee_spice, 
                                          obstime=Time(date_earth).isot, 
                                          representation_type='cartesian')
    
    # Return the HEE coordinates of the Earth
    return earth_hee

In [5]:
def get_so_hee(date_so):
    """
    Get the coordinates of Solar Orbiter and then return them in
    Heliocentric Earth Ecliptic (HEE) coordinates.
    """

    # Observing time (to get the SOLO coordinates)
    et_solo = spice.datetime2et(date_so)

    # Obtain the coordinates of Solar Orbiter
    solo_hee_spice = spice.spkpos('SOLO', et_solo, 'SOLO_HEE_NASA', 'NONE', 'SUN')[0]
    solo_hee_spice = solo_hee_spice * u.km

    # Convert the coordinates to HEE
    solo_hee = HeliocentricEarthEcliptic(solo_hee_spice, 
                                         obstime=Time(date_so).isot, 
                                         representation_type='cartesian')
    
    # Return the HEE coordinates of Solar Orbiter
    return solo_hee

### Functions to import the different flare lists

In [23]:
def import_stix(path_csv, check_test=False, earth_UT=False):
    '''
    Reads the csv file containing the STIX flare locations
    (obtained using: https://github.com/hayesla/stix_flarelist_science)
    Returns a pandas dataframe with the flares, the peak times and
    the flare locations in Heliographic Stonyhurst coordinates

    Parameters
    ----------
    path_csv : str
        Path to the csv file containing the STIX flare locations. It can be
        a list of several csv files.
    
    check_test : bool, optional (default=False)
        If True, only the flares that passed the location test are returned.

    earth_UT : bool, optional (default=False)
        If True, the peak times are returned in Earth time. If False, the
        peak times are returned in Solar Orbiter time.
    '''


    # If path_csv is a list, we need to concatenate all the csv files
    if type(path_csv) == list:
        df = pd.DataFrame()
        n_csv_files = len(path_csv)
        for i in range(n_csv_files):
            df = pd.concat([df,pd.read_csv(path_csv[i])], ignore_index=True)
    else:
        df = pd.read_csv(path_csv)
    
    # Sort the STIX dataframes by the start time of the flares
    df.sort_values(by='peak_UTC', inplace=True)
    df.reset_index(inplace=True, drop=True)

    # If check_test is True, remove the failed ones
    if check_test:
        df = df[df['flare location test']==True]
        df.reset_index(inplace=True, drop=True)
    
    # Save the columns of interest in lists from df
    start_times = pd.to_datetime(df['start_UTC'].tolist()) # SO time
    peak_times  = pd.to_datetime(df['peak_UTC'].tolist()) # SO time
    end_times   = pd.to_datetime(df['end_UTC'].tolist()) # SO time
    x_solo_hpc = df['x'].tolist() # arcsec, SO frame
    y_solo_hpc = df['y'].tolist() # arcsec, SO frame
    
    # [Status 30-Jun-2023]: light travel time correction not yet included in the flare list
    # Correct the all times for the light travel time
    if earth_UT:
        ltt_corr = pd.to_timedelta(df['ltt_corr'].tolist(), unit='s')
        start_times += ltt_corr
        peak_times  += ltt_corr
        end_times   += ltt_corr

    # Transform STIX coordinates in heliographic Stonyhurst [(lon, lat) in (deg, deg)]
    lon_hgs = []
    lat_hgs = []
    for i in range(len(x_solo_hpc)):

        # Get the HEE coordinates of the Earth at the time of the flare
        #earth_hee = get_earth_hee(peak_times[i])

        # Get the HEE coordinates of Solar Orbiter at the time of the flare
        solo_hee = get_so_hee(peak_times[i])

        # Transform the coordinates of the flare from SO hpc to heliographic Stonyhurst
        stix_hgc_stony = so2hgs([x_solo_hpc[i], y_solo_hpc[i]], peak_times[i], solo_hee)
        
        # Save the coordinates
        lon_hgs.append(stix_hgc_stony.lon.value)
        lat_hgs.append(stix_hgc_stony.lat.value)

    return df, start_times, peak_times, end_times, lon_hgs, lat_hgs

In [32]:
def import_hinode(path_csv):
    '''
    Reads the csv file containing the Hinode flare locations
    (exported from: https://hinode.isee.nagoya-u.ac.jp/flare_catalogue/)
    Returns a pandas dataframe with the flares, the peak times and
    the flare locations in Heliographic Stonyhurst coordinates

    Parameters
    ----------
    path_csv : str
        Path to the csv file containing the Hinode flare locations. It can be
        a list of several csv files.
    '''

    # If path_csv is a list, we need to concatenate all the csv files
    if type(path_csv) == list:
        df = pd.DataFrame()
        n_csv_files = len(path_csv)
        for i in range(n_csv_files):
            df = pd.concat([df,pd.read_csv(path_csv[i])], ignore_index=True)
    else:
        df = pd.read_csv(path_csv)

    # Sort the Hinode dataframes by the start time of the flares
    df.sort_values(by='start', inplace=True)
    df.reset_index(inplace=True, drop=True)

    # Remove the flares that have 0 in all XRT, EIS and SOT columns
    df = df[(df['XRT']!=0) | (df['EIS']!=0) | (df['SP']!=0) | (df['FG']!=0)]

    # Save the columns of interest in lists from df
    peak_times   = pd.to_datetime(df['peak'].tolist()) # Earth time
    ar_location = df['AR location'].tolist() # Stonyhurst heliographic coord

    # Transform Hinode coordinates in heliographic Stonyhurst [(lon, lat) in (deg, deg)]
    lon_hgs = []
    lat_hgs = []
    for i in range(len(ar_location)):

        sign_ns = 1
        sign_ew = 1
        if ar_location[i][0] == 'S': sign_ns = -1
        if ar_location[i][3] == 'E': sign_ew = -1
        coord_ns = float(ar_location[i][1:3])
        coord_ew = float(ar_location[i][4:6])
        lon = (sign_ew * coord_ew) #* u.deg
        lat = (sign_ns * coord_ns) #* u.deg

        # Convert the coordinates from Stonyhurst to HPC
        #coord = SkyCoord(lon*u.deg, lat*u.deg, frame=HeliographicStonyhurst, obstime=xrt_peak_time[i])
        #coord_hpc = coord.transform_to(Helioprojective(observer='earth', obstime=xrt_peak_time[i]))
        
        # Save the coordinates
        lon_hgs.append(lon)
        lat_hgs.append(lat)
        
    return df, peak_times, lon_hgs, lat_hgs

### Main script

In [8]:
# Path to the csv file containing the STIX locations
#path_csv_stix_flares = '/home/afbattaglia/Documents/ETHZ/PhD/Flares/Davos-Visit/stix-flare-locations/flare_location_df_first_pass_duplicates_v2.csv'
path_csv_stix_flares = '/home/afbattaglia/Documents/ETHZ/PhD/Codes/Python/test_flare-list/full-flarelist-with-paths-and-locations_no-NaNs.csv'

# Path to the folder containing the csv files with the Hinode locations
folder_csv_hinode_flares = '/home/afbattaglia/Documents/ETHZ/PhD/Flares/Davos-Visit/stix-flare-locations/hinode/'

# SPICE kernel
# Please visit: https://www.cosmos.esa.int/web/spice/solar_orbiter
path_spice = '/home/afbattaglia/Software/spice-kernels/solar-orbiter/kernels/mk'

# Threshold of the distance between the STIX and Hinode locations
# to consider them as the same flare
threshold_loc = 15   # degrees

# Time margin to consider the STIX and Hinode times as the same, 
# given in pandas.Timedelta format
threshold_time = pd.Timedelta('1 hour')

In [16]:
# Find all csv filed contained in the fodler_csv_hinode_flares
list_csv_hinode_flares = []
for file in os.listdir(folder_csv_hinode_flares):
    if file.endswith(".csv"):
        list_csv_hinode_flares.append(folder_csv_hinode_flares+file)
list_csv_hinode_flares

['/home/afbattaglia/Documents/ETHZ/PhD/Flares/Davos-Visit/stix-flare-locations/hinode/Hinode-Flare-Catalogue_2021.csv',
 '/home/afbattaglia/Documents/ETHZ/PhD/Flares/Davos-Visit/stix-flare-locations/hinode/Hinode-Flare-Catalogue_2022.csv']

In [14]:
# Load the SPICE kernel
load_SPICE(path_spice)


SPICE kernels loaded correctly



In [33]:
# Import the flare lists
df_stix, start_times_stix, peak_times_stix, end_times_stix, stix_lon_hgs, stix_lat_hgs = import_stix(path_csv_stix_flares, check_test=False, earth_UT=False)
df, peak_times, lon_hgs, lat_hgs = import_hinode(list_csv_hinode_flares)

##### Compare times and locations of the different instruments

In [34]:
# Select all the XRT flares that are within the STIX time window and location
combined_flare_list = pd.DataFrame()
for i in range(len(peak_times)):
    
    # First of all, we need to find the index of the STIX flare that is
    # closest to the Hinode flare
    time_diff = abs(peak_times[i] - peak_times_stix)
    idx_stix = np.where(time_diff == np.min(time_diff))[0][0]

    # Then, we need to check if the start and end times of the other instrument's flare 
    # are within the STIX time window plus the time margin
    check = False
    if (peak_times[i] >= start_times_stix[idx_stix]   - threshold_time) and (peak_times[i] <= end_times_stix[idx_stix]   + threshold_time): check = True
    if (peak_times[i] >= start_times_stix[idx_stix-1] - threshold_time) and (peak_times[i] <= end_times_stix[idx_stix-1] + threshold_time): check = True
    if (peak_times[i] >= start_times_stix[idx_stix+1] - threshold_time) and (peak_times[i] <= end_times_stix[idx_stix+1] + threshold_time): check = True
    
    if check:
            
        # Now check if the location of the flare is within the
        # threshold_loc from the STIX flare in heliographic Stonyhurst coordinates
        if (abs(lon_hgs[i] - stix_lon_hgs[idx_stix]) <= threshold_loc) and (abs(lat_hgs[i] - stix_lat_hgs[idx_stix]) <= threshold_loc):
            # Append the i-th element to xrt_stix_flares
            combined_flare_list = combined_flare_list._append(df.iloc[i])

# Reset the index of the combined flare list
combined_flare_list.reset_index(inplace=True, drop=True)

In [35]:
combined_flare_list

,Event number,start,peak,end,AR location,X-ray class,FG,SP,XRT,EIS,DARTS,RHESSI,Suzaku/WAM,NoRH
0,176310,2021/04/24 15:35,2021/04/24 15:59,2021/04/24 16:10,S21E12,B2.0,0,0,65,0,NaN,no,NaN,NaN
1,176320,2021/04/24 16:45,2021/04/24 16:53,2021/04/24 16:58,S21E12,B6.9,0,0,35,0,NaN,no,NaN,NaN
2,176340,2021/04/24 17:33,2021/04/24 17:37,2021/04/24 17:42,S21E11,B1.7,0,0,25,0,NaN,no,NaN,NaN
3,176930,2021/05/05 22:25,2021/05/05 22:31,2021/05/05 22:38,N15E88,B1.2,0,0,42,0,NaN,no,NaN,NaN
4,176940,2021/05/05 22:43,2021/05/05 22:48,2021/05/05 22:54,N17E88,B1.1,0,0,35,0,NaN,no,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,204720,2022/05/11 18:13,2022/05/11 18:58,2022/05/11 19:27,S17W89,M2.6,0,0,284,0,NaN,no,NaN,NaN
268,205760,2022/05/26 03:27,2022/05/26 03:42,2022/05/26 03:52,N19W89,C2.9,0,0,5,0,NaN,no,NaN,NaN
269,207380,2022/06/24 18:37,2022/06/24 18:44,2022/06/24 18:51,S17W79,C1.1,0,0,30,0,NaN,no,NaN,NaN
270,207500,2022/06/26 03:10,2022/06/26 03:21,2022/06/26 03:29,N22W74,C1.5,0,0,62,0,NaN,no,NaN,NaN
